# 🏃‍♂️ Criador de Plano de Treino Personalizado - Interativo

## Bem-vindo ao seu assistente de treino inteligente!

Este notebook usa **widgets interativos** para criar seu plano de treino personalizado.

**✨ Novidades:**
- 🎨 Interface visual com sliders, dropdowns e campos de texto
- 📊 Visualizações automáticas com gráficos
- 📄 Exportação em PDF com todo o plano
- 🚀 Mais fácil de usar - sem precisar entender código!

---

## 📦 Instalação de Dependências (Google Colab)

Se você estiver usando o Google Colab, execute a célula abaixo primeiro:

In [ ]:
# Instalar dependências necessárias (apenas no Google Colab)
try:
    import google.colab
    IN_COLAB = True
    print("🌐 Detectado Google Colab")
    print("📦 Instalando dependências...")
    !pip install -q ipywidgets reportlab matplotlib
    print("✅ Dependências instaladas!")
except:
    IN_COLAB = False
    print("💻 Ambiente local detectado")

---

## 🎯 Escolha o Modo de Criação

Temos dois modos disponíveis:

1. **🚀 Modo Simples**: Criação rápida com informações básicas
2. **⚙️ Modo Completo**: Personalização total com zonas de treino, lesões, etc.

Execute uma das células abaixo de acordo com sua preferência:

### 🚀 MODO SIMPLES - Criação Rápida

Execute esta célula para criar um plano rapidamente:

In [ ]:
from notebook_widgets import create_simple_plan_widgets

# Criar interface simples
widgets_simples = create_simple_plan_widgets()

### ⚙️ MODO COMPLETO - Personalização Total

Execute esta célula para criar um plano totalmente personalizado:

In [ ]:
from notebook_widgets import create_complete_plan_widgets

# Criar interface completa
widgets_completo = create_complete_plan_widgets()

---

## 📊 Visualizar Plano Gerado

Após gerar o plano usando os widgets acima, execute as células abaixo para diferentes visualizações:

### 📋 Resumo Visual do Plano

In [ ]:
# Pegar o plano gerado
try:
    if 'widgets_completo' in locals() and widgets_completo and widgets_completo.plan:
        plan = widgets_completo.plan
    elif 'widgets_simples' in locals() and widgets_simples and widgets_simples.plan:
        plan = widgets_simples.plan
    else:
        print("⚠️  Nenhum plano gerado ainda. Execute uma das células acima primeiro!")
        plan = None
    
    if plan:
        print("\n" + "="*70)
        print("📋 RESUMO DO SEU PLANO DE TREINO")
        print("="*70)
        plan.print_visual()
except Exception as e:
    print(f"❌ Erro: {e}")

### 📅 Ver Todas as Semanas Detalhadas

In [ ]:
if plan:
    plan.print_visual(show_all_weeks=True)
else:
    print("⚠️  Gere um plano primeiro!")

### 📊 Gráficos de Visualização

Veja gráficos bonitos do seu plano:

In [ ]:
if plan:
    try:
        from plot_utils import plot_weekly_volume, plot_zone_distribution_stacked, print_zone_summary
        import matplotlib.pyplot as plt
        
        # Gráfico de volume semanal
        print("📊 Volume Semanal (km)\n")
        fig, ax = plot_weekly_volume(plan)
        plt.show()
        
        # Gráfico de distribuição de zonas
        if hasattr(plan, 'training_zones') and plan.training_zones:
            print("\n📊 Distribuição de Zonas por Semana\n")
            fig, ax = plot_zone_distribution_stacked(plan)
            plt.show()
            
            # Resumo textual
            print_zone_summary(plan)
    except ImportError:
        print("⚠️  matplotlib não disponível. Instale com: pip install matplotlib")
    except Exception as e:
        print(f"❌ Erro ao gerar gráficos: {e}")
else:
    print("⚠️  Gere um plano primeiro!")

### 📊 Tabela de Zonas de Treino

Veja suas zonas de treino personalizadas (se você informou tempos de prova):

In [ ]:
if plan and hasattr(plan, 'training_zones') and plan.training_zones:
    print(plan.training_zones.to_table())
elif plan:
    print("ℹ️  Este plano não tem zonas de treino calculadas.")
    print("   Para ter zonas personalizadas, preencha seus tempos recentes de prova.")
else:
    print("⚠️  Gere um plano primeiro!")

---

## 💾 Salvar Plano

Salve seu plano em diferentes formatos:

### 📄 Salvar como JSON

In [ ]:
if plan:
    # Nome do arquivo baseado no plano
    filename_json = plan.name.replace(" ", "_").replace("/", "_") + ".json"
    plan.save_to_file(filename_json)
    
    print(f"✅ Plano salvo em: {filename_json}")
    print(f"\n💡 Você pode carregar este plano depois com:")
    print(f"   from running_plan import RunningPlan")
    print(f"   plan = RunningPlan.load_from_file('{filename_json}')")
else:
    print("⚠️  Gere um plano primeiro!")

### 📄 Exportar como PDF

**✨ NOVO!** Exporte seu plano completo em PDF com gráficos:

In [ ]:
if plan:
    from pdf_export import save_plan_as_pdf
    
    # Gerar PDF com gráficos
    pdf_file = save_plan_as_pdf(plan, include_graphs=True)
    
    if pdf_file:
        print(f"\n📥 Para baixar o PDF:")
        if IN_COLAB:
            from google.colab import files
            print(f"   Clique no link abaixo para fazer download:")
            files.download(pdf_file)
        else:
            print(f"   O arquivo está em: {pdf_file}")
else:
    print("⚠️  Gere um plano primeiro!")

### 🔄 Upload para Intervals.icu

**✨ NOVO!** Envie seu plano diretamente para sua conta Intervals.icu:

**Configuração (primeira vez):**
1. Acesse https://intervals.icu/ → Settings → Developer → API Key
2. Copie sua API Key completa (formato: `athlete_12345:chave_longa`)
3. Execute a célula abaixo para configurar

In [ ]:
# CONFIGURAÇÃO INICIAL - Execute apenas na primeira vez
# Substitua os valores abaixo pelas suas credenciais reais

from intervals_integration import create_config_file

# ⚠️  SUBSTITUA ESTES VALORES PELAS SUAS CREDENCIAIS:
# minha_api_key = "athlete_12345:sua_chave_completa_aqui"
# meu_athlete_id = "12345"

# Descomente as linhas abaixo após preencher suas credenciais:
# create_config_file(minha_api_key, meu_athlete_id)

print("💡 Instruções:")
print("1. Descomente as 3 linhas acima")
print("2. Substitua os valores com suas credenciais reais")
print("3. Execute esta célula")
print("4. Comente as linhas novamente para segurança")

**Testar Conexão:**

In [ ]:
# Testar se a configuração está correta
from intervals_integration import IntervalsUploader

uploader = IntervalsUploader()
uploader.test_connection()

**Fazer Upload do Plano:**

In [ ]:
# Upload do plano para Intervals.icu
if plan:
    from intervals_integration import IntervalsUploader
    
    uploader = IntervalsUploader()
    success = uploader.upload_plan(plan)
    
    if success:
        print("\n✅ Seu plano está disponível no Intervals.icu!")
        print("📱 Agora você pode acompanhar seus treinos pelo app ou web")
else:
    print("⚠️  Gere um plano primeiro!")

---

## 🎉 Parabéns!

Seu plano de treino personalizado está pronto! 

### 💡 Próximos Passos:

1. ✅ Baixe o PDF gerado
2. 📱 Imprima ou salve no seu celular
3. 🏃 Siga o plano consistentemente
4. 👂 Escute seu corpo e ajuste quando necessário
5. 🎯 Boa sorte na sua prova!

### 📋 Lembre-se:

- **Consistência** é mais importante que intensidade
- **Descanso** é parte essencial do treino
- **Hidratação** e **nutrição** são fundamentais
- **Ouça seu corpo** - descanse se necessário

---

**🏃‍♂️ Bons treinos e sucesso na sua jornada! 🎉**